In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
180,ABW,North America,Aruba,2020-09-14,3046.0,52.0,85.286,18.0,0.0,0.571,...,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29,NaN
439,AFG,Asia,Afghanistan,2020-09-14,38716.0,75.0,45.429,1420.0,0.0,1.143,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
616,AGO,Africa,Angola,2020-09-14,3335.0,56.0,57.143,132.0,1.0,2.143,...,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15,0.581


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-13,France,382338.0,0.0,30991.0,0.0,0.0,0.0
0,2020-09-13,China,90192.0,0.0,4739.0,0.0,0.0,0.0
0,2020-09-13,Italy,287720.0,0.0,35631.0,0.0,0.0,0.0
0,2020-09-13,Spain,573783.0,0.0,29790.0,0.0,0.0,0.0
0,2020-09-13,United States,6562728.0,0.0,195025.0,0.0,0.0,0.0
0,2020-09-13,World,29263167.0,0.0,929725.0,0.0,0.0,0.0
0,2020-09-13,United Kingdom,367906.0,0.0,41661.0,0.0,0.0,0.0
0,2020-09-13,Germany,260910.0,0.0,9359.0,0.0,0.0,0.0
0,2020-09-13,Iran,403706.0,0.0,23270.0,0.0,0.0,0.0
0,2020-09-13,Turkey,290977.0,0.0,7044.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-09-14,France,382338.0,381094.0,30991.0,30916.0,1244.0,75.0
0,2020-09-14,China,90192.0,90197.0,4739.0,4734.0,-5.0,5.0
0,2020-09-14,Italy,287720.0,287753.0,35631.0,35610.0,-33.0,21.0
0,2020-09-13,Spain,573783.0,566326.0,29790.0,29747.0,7457.0,43.0
0,2020-09-14,United States,6562728.0,6519979.0,195025.0,194079.0,42749.0,946.0
0,2020-09-14,World,29263167.0,29049134.0,929725.0,924706.0,214033.0,5019.0
0,2020-09-14,United Kingdom,367906.0,368504.0,41661.0,41628.0,-598.0,33.0
0,2020-09-14,Germany,260910.0,260355.0,9359.0,9350.0,555.0,9.0
0,2020-09-14,Iran,403706.0,402029.0,23270.0,23157.0,1677.0,113.0
0,2020-09-14,Turkey,290977.0,291162.0,7044.0,7056.0,-185.0,-12.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")